In [86]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd
import pyautogui
pyautogui.PAUSE = 2.5

In [90]:
#Creating a list of all schools to extract near companies
df = pd.read_excel('EFG - ENDEREÇO.xlsx')
schools_urls_list = df['google_maps_url'].to_list()
schools_names_list = df['name'].to_list()

In [89]:
def get_company_links(list_of_google_maps_urls):
    # This is the list where we'll capture the companies' names and addresses
    links_list = []
    for url in list_of_google_maps_urls:
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(url)
        driver.implicitly_wait(10)
        action = ActionChains(driver)
        sleep(3)
        #Clicking on nearby button
        try:
            nearby_button = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[3]/div/div[1]")
            driver.implicitly_wait(10)
            nearby_button.click()
        except:
            pass

        #Writing "company" and pressing enter
        sleep(2)
        driver.implicitly_wait(10)
        pyautogui.typewrite('company\n')
        sleep(2)

        for x in range(1,6):
            if x==1:
                sleep(2)
                #Move Mouse to companies
                pyautogui.moveTo(175, 312)
                sleep(3)

                #Scroll maximum
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)

                # Capturing the names and addresses of the companies and adding them to our list 'competition' :
                result_names = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[4]')

                soup = BeautifulSoup(result_names.get_attribute('outerHTML'),'html5lib')
                for href in soup('a',{"aria-label": True}):
                    links_list.append(href['href'])
            else:
                #Click "more" button
                try:
                    next_button = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[2]/div/div[1]/div/button[2]/img")
                    driver.implicitly_wait(10)
                    next_button.click()
                except:
                    pass
                sleep(2)
                #Move Mouse to companies
                pyautogui.moveTo(175, 312)
                sleep(3)

                #Scroll maximum
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)
                pyautogui.scroll(-4000)

                # Capturing the names and addresses of the companies and adding them to our list 'competition' :
                result_names = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[4]')

                soup = BeautifulSoup(result_names.get_attribute('outerHTML'),'html5lib')
                for href in soup('a',{"aria-label": True}):
                    links_list.append(href['href'])
        driver.quit()
    return links_list

In [91]:
links = get_company_links(schools_urls_list)

In [104]:
df_links_final = pd.DataFrame(links)
df_links_final.columns = ['link']
df_links_final.to_csv('efg_companies_link.csv', index=False, sep="|")

In [157]:
def get_company_details(list_of_urls_from_google_maps):
    #creating data_frame do receive data
    df_companyinfo = pd.DataFrame(columns=[
                                            'company_google_maps_url',
                                            'company_name',
                                            'company_avg_rating',
                                            'company_number_of_ratings',
                                            'company_category',
                                            'company_address',
                                            'company_website',
                                            'company_phone'])
    for url in list_of_urls_from_google_maps:
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(url)
        driver.implicitly_wait(10)
        action = ActionChains(driver)

        sleep(3)
        try:
            driver.implicitly_wait(10)
            company_name = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]").text
        except:
            company_name = ''

        try:
            driver.implicitly_wait(10)
            company_avg_rating = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span").text
        except:
            company_avg_rating = ''

        try:
            driver.implicitly_wait(10)
            company_number_of_ratings = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button").text
        except:
            company_number_of_ratings = ''

        try:
            driver.implicitly_wait(10)
            company_category = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button").text
        except:
            company_category = ''
        
        try:
            driver.implicitly_wait(10)
            company_address = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[7]/div[1]/button/div[1]/div[2]/div[1]").text
        except:
            company_address = ''

        try:
            driver.implicitly_wait(10)
            company_website = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[7]/div[3]/button/div[1]/div[2]/div[1]").text
        except:
            company_website = ''

        try:
            driver.implicitly_wait(10)
            company_phone = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[7]/div[4]/button/div[1]/div[2]/div[1]").text
        except:
            company_phone = ''
        
        driver.quit()

        #Adding some info to df
        df_companyinfo = df_companyinfo.append({
                'company_google_maps_url':url,
                'company_name':company_name,
                'company_avg_rating':company_avg_rating,
                'company_number_of_ratings':company_number_of_ratings,
                'company_category':company_category,
                'company_address':company_address,
                'company_website':company_website,
                'company_phone':company_phone}, ignore_index=True)
        if len(df_companyinfo) == 5:
            df_companyinfo.to_csv('companies_details_len100.csv',index=False,sep="|", encoding='windows-1252')
        if len(df_companyinfo) == 100:
            df_companyinfo.to_csv('companies_details_len100.csv',index=False,sep="|", encoding='windows-1252')
        if len(df_companyinfo) == 200:
            df_companyinfo.to_csv('companies_details_len200.csv',index=False,sep="|", encoding='windows-1252')
        if len(df_companyinfo) == 300:
            df_companyinfo.to_csv('companies_details_len300.csv',index=False,sep="|", encoding='windows-1252')
        if len(df_companyinfo) == 400:
            df_companyinfo.to_csv('companies_details_len300.csv',index=False,sep="|", encoding='windows-1252')
        if len(df_companyinfo) == 500:
            df_companyinfo.to_csv('companies_details_len300.csv',index=False,sep="|", encoding='windows-1252')
    return df_companyinfo

In [158]:
df_final = get_company_details(links)